In [86]:
import pandas as pd
import numpy as np
traindata = pd.read_csv('train.csv')
testdata = pd.read_csv('test.csv')
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [87]:
traindata.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


age has missing data

In [88]:
traindata['Age'].fillna(traindata['Age'].median(), inplace=True)

<ipython-input-88-8c8683abe27f>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  traindata['Age'].fillna(traindata['Age'].median(), inplace=True)


In [89]:
traindata['Embarked'].fillna(traindata['Embarked'].mode()[0], inplace=True)

<ipython-input-89-2f6ca5a68816>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  traindata['Embarked'].fillna(traindata['Embarked'].mode()[0], inplace=True)


In [90]:
traindata['Sex']=traindata['Sex'].map({'male':0, 'female':1})

In [91]:
traindata.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [92]:
x=traindata.drop(['PassengerId','Name','Survived','Ticket','Parch','SibSp','Fare','Cabin'], axis=1)
y=traindata['Survived']

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [94]:
print(x_train.dtypes)

Pclass        int64
Sex           int64
Age         float64
Embarked     object
dtype: object


In [95]:
x_train = pd.get_dummies(x_train, columns=['Embarked'], drop_first=True)

In [96]:
print(x_train.dtypes)

Pclass          int64
Sex             int64
Age           float64
Embarked_Q       bool
Embarked_S       bool
dtype: object


In [97]:
model=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, random_state=1)

In [98]:
testdata=pd.read_csv('test.csv')

In [99]:
for col in ['Embarked_Q','Embarked_S']:
    if col not in testdata.columns:
      x_test[col] = 0

In [100]:
testdata['Sex']=testdata['Sex'].map({'male':0, 'female':1})
# Ensure both 'Embarked_Q' and 'Embarked_S' exist
for col in ['Embarked_Q', 'Embarked_S']:
    if col not in testdata.columns:
        testdata[col] = 0  # Add the missing column with default value 0

# Now select the features
X_test = testdata[['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S']]

In [101]:
print (testdata.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Embarked_Q       int64
Embarked_S       int64
dtype: object


In [102]:
print (x_test.columns)

Index(['Pclass', 'Sex', 'Age', 'Embarked', 'Embarked_Q', 'Embarked_S'], dtype='object')


In [103]:
x_test=testdata[['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S']]

In [104]:
print(x_test.columns)

Index(['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S'], dtype='object')


In [105]:
predictions=model.predict(x_test)

In [106]:
predictions=model.predict(x_test)

In [107]:
submission= pd.DataFrame({'PassengerId': testdata['PassengerId'], 'Survived': predictions})

In [108]:
submission.to_csv('submission.csv', index=False)